<a href="https://colab.research.google.com/github/sameersahu473/comp1/blob/main/third.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
! pip install kaggle

In [68]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [69]:
! cp kaggle.json ~/.kaggle/

In [70]:
! chmod 600 ~/.kaggle/kaggle.json


In [71]:
!kaggle competitions download -c cascade-cup-22

test.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [72]:
!unzip train.csv.zip

Archive:  train.csv.zip
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv               


In [73]:
!unzip test.csv.zip

Archive:  test.csv.zip
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.csv                


In [74]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [75]:
df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [76]:
df.head()

,order_time,order_id,order_date,allot_time,accept_time,pickup_time,delivered_time,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,cancelled,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time,cancelled_time
0,2021-01-26 02:21:35,556753,2021-01-26 00:00:00,2021-01-26 02:21:59,2021-01-26 02:22:08,2021-01-26 02:32:51,2021-01-26 02:49:47,11696,1.5666,2.65,46.0,46.0,0,0.0,621.0,NaN,NaN,NaN,NaN,NaN
1,2021-01-26 02:33:16,556754,2021-01-26 00:00:00,2021-01-26 02:33:57,2021-01-26 02:34:45,2021-01-26 02:50:25,2021-01-26 03:11:15,18117,2.5207,2.76,8.0,8.0,0,0.0,105.0,NaN,NaN,NaN,3.266667,NaN
2,2021-01-26 02:39:49,556755,2021-01-26 00:00:00,2021-01-26 02:39:57,2021-01-26 02:40:13,2021-01-26 02:56:00,2021-01-26 03:12:46,18623,2.2074,4.80,1.0,1.0,0,0.0,66.0,NaN,NaN,NaN,9.816667,NaN
3,2021-01-26 02:47:53,556756,2021-01-26 00:00:00,2021-01-26 02:48:25,2021-01-26 02:49:06,2021-01-26 03:21:51,2021-01-26 03:41:05,15945,2.1894,6.38,1.0,1.0,0,0.0,127.0,NaN,NaN,NaN,17.533333,NaN
4,2021-01-26 03:06:30,556757,2021-01-26 00:00:00,2021-01-26 03:07:21,2021-01-26 03:07:57,2021-01-26 03:31:38,2021-01-26 04:00:15,17589,2.7870,4.01,34.0,34.0,0,0.0,84.0,NaN,NaN,NaN,1.350000,NaN


In [77]:
df_test.head()

,order_time,order_id,order_date,allot_time,accept_time,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time
0,2021-02-06 10:03:24,130231,2021-02-06 00:00:00,2021-02-06 10:03:49,2021-02-06 10:04:15,12884,1.6585,4.54,216.0,215.0,1.0,747.0,NaN,NaN,NaN,273.400000
1,2021-02-06 10:03:26,130232,2021-02-06 00:00:00,2021-02-06 10:03:27,2021-02-06 10:03:36,3541,2.0709,5.84,52.0,52.0,0.0,75.0,NaN,NaN,NaN,252.100000
2,2021-02-06 10:03:27,130233,2021-02-06 00:00:00,2021-02-06 10:04:14,2021-02-06 10:05:34,603,1.3884,0.99,289.0,289.0,0.0,2214.0,NaN,NaN,NaN,241.383333
3,2021-02-06 10:03:29,130234,2021-02-06 00:00:00,2021-02-06 10:03:30,2021-02-06 10:03:53,3414,1.9039,2.59,125.0,122.0,3.0,1020.0,NaN,NaN,NaN,291.933333
4,2021-02-06 10:03:35,130235,2021-02-06 00:00:00,2021-02-06 10:03:43,2021-02-06 10:04:43,1426,0.8275,0.94,352.0,350.0,2.0,7284.0,NaN,NaN,NaN,247.133333


In [78]:
df['rider_id'].value_counts()

237      228
190      213
11998    209
170      203
853      200
        ... 
9339       1
20693      1
16603      1
20692      1
4895       1
Name: rider_id, Length: 19537, dtype: int64

In [79]:
df.shape

(450000, 20)

In [80]:
df.isnull().sum()

order_time                   0
order_id                     0
order_date                   0
allot_time                   0
accept_time                157
pickup_time               2421
delivered_time            5218
rider_id                     0
first_mile_distance          0
last_mile_distance           0
alloted_orders           16948
delivered_orders         17341
cancelled                    0
undelivered_orders       17341
lifetime_order_count        53
reassignment_method     436256
reassignment_reason     436247
reassigned_order        436247
session_time              3675
cancelled_time          444782
dtype: int64

In [81]:
df = df.drop(columns=['order_id','pickup_time','delivered_time'],axis =1)

In [82]:
df.head()

,order_time,order_date,allot_time,accept_time,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,cancelled,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time,cancelled_time
0,2021-01-26 02:21:35,2021-01-26 00:00:00,2021-01-26 02:21:59,2021-01-26 02:22:08,11696,1.5666,2.65,46.0,46.0,0,0.0,621.0,NaN,NaN,NaN,NaN,NaN
1,2021-01-26 02:33:16,2021-01-26 00:00:00,2021-01-26 02:33:57,2021-01-26 02:34:45,18117,2.5207,2.76,8.0,8.0,0,0.0,105.0,NaN,NaN,NaN,3.266667,NaN
2,2021-01-26 02:39:49,2021-01-26 00:00:00,2021-01-26 02:39:57,2021-01-26 02:40:13,18623,2.2074,4.80,1.0,1.0,0,0.0,66.0,NaN,NaN,NaN,9.816667,NaN
3,2021-01-26 02:47:53,2021-01-26 00:00:00,2021-01-26 02:48:25,2021-01-26 02:49:06,15945,2.1894,6.38,1.0,1.0,0,0.0,127.0,NaN,NaN,NaN,17.533333,NaN
4,2021-01-26 03:06:30,2021-01-26 00:00:00,2021-01-26 03:07:21,2021-01-26 03:07:57,17589,2.7870,4.01,34.0,34.0,0,0.0,84.0,NaN,NaN,NaN,1.350000,NaN


In [83]:
df_test = df_test.drop(['order_id'],axis=1)

In [84]:
df.isnull().sum()

order_time                   0
order_date                   0
allot_time                   0
accept_time                157
rider_id                     0
first_mile_distance          0
last_mile_distance           0
alloted_orders           16948
delivered_orders         17341
cancelled                    0
undelivered_orders       17341
lifetime_order_count        53
reassignment_method     436256
reassignment_reason     436247
reassigned_order        436247
session_time              3675
cancelled_time          444782
dtype: int64

In [85]:
df['cancelled'].value_counts()

0    444782
1      5218
Name: cancelled, dtype: int64

In [86]:
for col in df.columns:  
    dataTypeSeries = df[col].dtypes
    print('Data type of '+ col+ ' column of Dataframe :')
    print(dataTypeSeries)

Data type of order_time column of Dataframe :
object
Data type of order_date column of Dataframe :
object
Data type of allot_time column of Dataframe :
object
Data type of accept_time column of Dataframe :
object
Data type of rider_id column of Dataframe :
int64
Data type of first_mile_distance column of Dataframe :
float64
Data type of last_mile_distance column of Dataframe :
float64
Data type of alloted_orders column of Dataframe :
float64
Data type of delivered_orders column of Dataframe :
float64
Data type of cancelled column of Dataframe :
int64
Data type of undelivered_orders column of Dataframe :
float64
Data type of lifetime_order_count column of Dataframe :
float64
Data type of reassignment_method column of Dataframe :
object
Data type of reassignment_reason column of Dataframe :
object
Data type of reassigned_order column of Dataframe :
float64
Data type of session_time column of Dataframe :
float64
Data type of cancelled_time column of Dataframe :
object


In [87]:
#creating the datetime column
df['allot_time'] = pd.to_datetime(df['allot_time'].str.strip(), format='%Y/%m/%d %H:%M:%S')
df_test['allot_time'] = pd.to_datetime(df_test['allot_time'].str.strip(), format='%Y/%m/%d %H:%M:%S')

In [88]:
df['accept_time'] = pd.to_datetime(df['accept_time'].str.strip(), format='%Y/%m/%d %H:%M:%S')
df_test['accept_time'] = pd.to_datetime(df_test['accept_time'].str.strip(), format='%Y/%m/%d %H:%M:%S')

In [89]:
for col in df.columns:  
    dataTypeSeries = df[col].dtypes
    print('Data type of '+ col+ ' column of Dataframe :')
    print(dataTypeSeries)

Data type of order_time column of Dataframe :
object
Data type of order_date column of Dataframe :
object
Data type of allot_time column of Dataframe :
datetime64[ns]
Data type of accept_time column of Dataframe :
datetime64[ns]
Data type of rider_id column of Dataframe :
int64
Data type of first_mile_distance column of Dataframe :
float64
Data type of last_mile_distance column of Dataframe :
float64
Data type of alloted_orders column of Dataframe :
float64
Data type of delivered_orders column of Dataframe :
float64
Data type of cancelled column of Dataframe :
int64
Data type of undelivered_orders column of Dataframe :
float64
Data type of lifetime_order_count column of Dataframe :
float64
Data type of reassignment_method column of Dataframe :
object
Data type of reassignment_reason column of Dataframe :
object
Data type of reassigned_order column of Dataframe :
float64
Data type of session_time column of Dataframe :
float64
Data type of cancelled_time column of Dataframe :
object


In [90]:
df['accept_to_allot'] = (df['accept_time'] - df['allot_time'])
df_test['accept_to_allot'] = (df_test['accept_time'] - df_test['allot_time'])

In [91]:
df.head()

,order_time,order_date,allot_time,accept_time,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,cancelled,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time,cancelled_time,accept_to_allot
0,2021-01-26 02:21:35,2021-01-26 00:00:00,2021-01-26 02:21:59,2021-01-26 02:22:08,11696,1.5666,2.65,46.0,46.0,0,0.0,621.0,NaN,NaN,NaN,NaN,NaN,0 days 00:00:09
1,2021-01-26 02:33:16,2021-01-26 00:00:00,2021-01-26 02:33:57,2021-01-26 02:34:45,18117,2.5207,2.76,8.0,8.0,0,0.0,105.0,NaN,NaN,NaN,3.266667,NaN,0 days 00:00:48
2,2021-01-26 02:39:49,2021-01-26 00:00:00,2021-01-26 02:39:57,2021-01-26 02:40:13,18623,2.2074,4.80,1.0,1.0,0,0.0,66.0,NaN,NaN,NaN,9.816667,NaN,0 days 00:00:16
3,2021-01-26 02:47:53,2021-01-26 00:00:00,2021-01-26 02:48:25,2021-01-26 02:49:06,15945,2.1894,6.38,1.0,1.0,0,0.0,127.0,NaN,NaN,NaN,17.533333,NaN,0 days 00:00:41
4,2021-01-26 03:06:30,2021-01-26 00:00:00,2021-01-26 03:07:21,2021-01-26 03:07:57,17589,2.7870,4.01,34.0,34.0,0,0.0,84.0,NaN,NaN,NaN,1.350000,NaN,0 days 00:00:36


In [92]:
for col in df.columns:  
    dataTypeSeries = df[col].dtypes
    print('Data type of '+ col+ ' column of Dataframe :')
    print(dataTypeSeries)

Data type of order_time column of Dataframe :
object
Data type of order_date column of Dataframe :
object
Data type of allot_time column of Dataframe :
datetime64[ns]
Data type of accept_time column of Dataframe :
datetime64[ns]
Data type of rider_id column of Dataframe :
int64
Data type of first_mile_distance column of Dataframe :
float64
Data type of last_mile_distance column of Dataframe :
float64
Data type of alloted_orders column of Dataframe :
float64
Data type of delivered_orders column of Dataframe :
float64
Data type of cancelled column of Dataframe :
int64
Data type of undelivered_orders column of Dataframe :
float64
Data type of lifetime_order_count column of Dataframe :
float64
Data type of reassignment_method column of Dataframe :
object
Data type of reassignment_reason column of Dataframe :
object
Data type of reassigned_order column of Dataframe :
float64
Data type of session_time column of Dataframe :
float64
Data type of cancelled_time column of Dataframe :
object
Data

In [93]:
df['accept_sec'] = df[:5]['accept_to_allot'].astype('timedelta64[s]')
df_test['accept_sec'] = df_test[:5]['accept_to_allot'].astype('timedelta64[s]')

In [94]:
df.head()

,order_time,order_date,allot_time,accept_time,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,cancelled,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time,cancelled_time,accept_to_allot,accept_sec
0,2021-01-26 02:21:35,2021-01-26 00:00:00,2021-01-26 02:21:59,2021-01-26 02:22:08,11696,1.5666,2.65,46.0,46.0,0,0.0,621.0,NaN,NaN,NaN,NaN,NaN,0 days 00:00:09,9.0
1,2021-01-26 02:33:16,2021-01-26 00:00:00,2021-01-26 02:33:57,2021-01-26 02:34:45,18117,2.5207,2.76,8.0,8.0,0,0.0,105.0,NaN,NaN,NaN,3.266667,NaN,0 days 00:00:48,48.0
2,2021-01-26 02:39:49,2021-01-26 00:00:00,2021-01-26 02:39:57,2021-01-26 02:40:13,18623,2.2074,4.80,1.0,1.0,0,0.0,66.0,NaN,NaN,NaN,9.816667,NaN,0 days 00:00:16,16.0
3,2021-01-26 02:47:53,2021-01-26 00:00:00,2021-01-26 02:48:25,2021-01-26 02:49:06,15945,2.1894,6.38,1.0,1.0,0,0.0,127.0,NaN,NaN,NaN,17.533333,NaN,0 days 00:00:41,41.0
4,2021-01-26 03:06:30,2021-01-26 00:00:00,2021-01-26 03:07:21,2021-01-26 03:07:57,17589,2.7870,4.01,34.0,34.0,0,0.0,84.0,NaN,NaN,NaN,1.350000,NaN,0 days 00:00:36,36.0


In [95]:
df_test.head()

,order_time,order_date,allot_time,accept_time,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time,accept_to_allot,accept_sec
0,2021-02-06 10:03:24,2021-02-06 00:00:00,2021-02-06 10:03:49,2021-02-06 10:04:15,12884,1.6585,4.54,216.0,215.0,1.0,747.0,NaN,NaN,NaN,273.400000,0 days 00:00:26,26.0
1,2021-02-06 10:03:26,2021-02-06 00:00:00,2021-02-06 10:03:27,2021-02-06 10:03:36,3541,2.0709,5.84,52.0,52.0,0.0,75.0,NaN,NaN,NaN,252.100000,0 days 00:00:09,9.0
2,2021-02-06 10:03:27,2021-02-06 00:00:00,2021-02-06 10:04:14,2021-02-06 10:05:34,603,1.3884,0.99,289.0,289.0,0.0,2214.0,NaN,NaN,NaN,241.383333,0 days 00:01:20,80.0
3,2021-02-06 10:03:29,2021-02-06 00:00:00,2021-02-06 10:03:30,2021-02-06 10:03:53,3414,1.9039,2.59,125.0,122.0,3.0,1020.0,NaN,NaN,NaN,291.933333,0 days 00:00:23,23.0
4,2021-02-06 10:03:35,2021-02-06 00:00:00,2021-02-06 10:03:43,2021-02-06 10:04:43,1426,0.8275,0.94,352.0,350.0,2.0,7284.0,NaN,NaN,NaN,247.133333,0 days 00:01:00,60.0


In [96]:
#creating the datetime column
df['order_time'] = pd.to_datetime(df['order_time'].str.strip(), format='%Y/%m/%d %H:%M:%S')
df_test['order_time'] = pd.to_datetime(df_test['order_time'].str.strip(), format='%Y/%m/%d %H:%M:%S')

In [97]:
df['order_to_allot'] = (df['accept_time'] - df['order_time'])
df_test['order_to_allot'] = (df_test['accept_time'] - df_test['order_time'])

In [98]:
df['order_sec'] = df[:5]['order_to_allot'].astype('timedelta64[s]')
df_test['order_sec'] = df_test[:5]['order_to_allot'].astype('timedelta64[s]')

In [99]:
df.head()

,order_time,order_date,allot_time,accept_time,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,cancelled,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time,cancelled_time,accept_to_allot,accept_sec,order_to_allot,order_sec
0,2021-01-26 02:21:35,2021-01-26 00:00:00,2021-01-26 02:21:59,2021-01-26 02:22:08,11696,1.5666,2.65,46.0,46.0,0,0.0,621.0,NaN,NaN,NaN,NaN,NaN,0 days 00:00:09,9.0,0 days 00:00:33,33.0
1,2021-01-26 02:33:16,2021-01-26 00:00:00,2021-01-26 02:33:57,2021-01-26 02:34:45,18117,2.5207,2.76,8.0,8.0,0,0.0,105.0,NaN,NaN,NaN,3.266667,NaN,0 days 00:00:48,48.0,0 days 00:01:29,89.0
2,2021-01-26 02:39:49,2021-01-26 00:00:00,2021-01-26 02:39:57,2021-01-26 02:40:13,18623,2.2074,4.80,1.0,1.0,0,0.0,66.0,NaN,NaN,NaN,9.816667,NaN,0 days 00:00:16,16.0,0 days 00:00:24,24.0
3,2021-01-26 02:47:53,2021-01-26 00:00:00,2021-01-26 02:48:25,2021-01-26 02:49:06,15945,2.1894,6.38,1.0,1.0,0,0.0,127.0,NaN,NaN,NaN,17.533333,NaN,0 days 00:00:41,41.0,0 days 00:01:13,73.0
4,2021-01-26 03:06:30,2021-01-26 00:00:00,2021-01-26 03:07:21,2021-01-26 03:07:57,17589,2.7870,4.01,34.0,34.0,0,0.0,84.0,NaN,NaN,NaN,1.350000,NaN,0 days 00:00:36,36.0,0 days 00:01:27,87.0


In [100]:
df['allot_sec'] = df['order_sec'] - df['accept_sec']
df_test['allot_sec'] = df_test['order_sec'] - df_test['accept_sec']

In [101]:
df_test = df_test.drop(['order_time','allot_time','accept_time','accept_to_allot','order_to_allot'],axis=1)
df= df.drop(['order_time','allot_time','accept_time','accept_to_allot','order_to_allot'],axis=1)

In [102]:
df['order_date'] = pd.to_datetime(df['order_date'].str.strip(), format='%Y/%m/%d %H:%M:%S')
df_test['order_date'] = pd.to_datetime(df_test['order_date'].str.strip(), format='%Y/%m/%d %H:%M:%S')

In [103]:
df['year'] = pd.DatetimeIndex(df['order_date']).year
df['month'] = pd.DatetimeIndex(df['order_date']).month


df_test['year'] = pd.DatetimeIndex(df_test['order_date']).year
df_test['month'] = pd.DatetimeIndex(df_test['order_date']).month

In [104]:
df.head()

,order_date,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,cancelled,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time,cancelled_time,accept_sec,order_sec,allot_sec,year,month
0,2021-01-26,11696,1.5666,2.65,46.0,46.0,0,0.0,621.0,NaN,NaN,NaN,NaN,NaN,9.0,33.0,24.0,2021,1
1,2021-01-26,18117,2.5207,2.76,8.0,8.0,0,0.0,105.0,NaN,NaN,NaN,3.266667,NaN,48.0,89.0,41.0,2021,1
2,2021-01-26,18623,2.2074,4.80,1.0,1.0,0,0.0,66.0,NaN,NaN,NaN,9.816667,NaN,16.0,24.0,8.0,2021,1
3,2021-01-26,15945,2.1894,6.38,1.0,1.0,0,0.0,127.0,NaN,NaN,NaN,17.533333,NaN,41.0,73.0,32.0,2021,1
4,2021-01-26,17589,2.7870,4.01,34.0,34.0,0,0.0,84.0,NaN,NaN,NaN,1.350000,NaN,36.0,87.0,51.0,2021,1


In [105]:
df.columns

Index(['order_date', 'rider_id', 'first_mile_distance', 'last_mile_distance',
       'alloted_orders', 'delivered_orders', 'cancelled', 'undelivered_orders',
       'lifetime_order_count', 'reassignment_method', 'reassignment_reason',
       'reassigned_order', 'session_time', 'cancelled_time', 'accept_sec',
       'order_sec', 'allot_sec', 'year', 'month'],
      dtype='object')

In [106]:
df['reassignment_reason'].value_counts()


Auto Reassignment basis Inaction. coreengine.tasks.repush_order_to_aa_bucket    7212
Reassignment Request from SE portal.                                            5300
Reassign                                                                        1241
Name: reassignment_reason, dtype: int64

In [107]:
df_test['reassignment_reason'].value_counts()

Auto Reassignment basis Inaction. coreengine.tasks.repush_order_to_aa_bucket    2593
Reassignment Request from SE portal.                                            1582
Reassign                                                                         460
Name: reassignment_reason, dtype: int64

In [108]:
df['reassignment_method'].value_counts()

auto      13383
manual      361
Name: reassignment_method, dtype: int64

In [109]:
df_test['reassignment_method'].value_counts()

auto      4531
manual     101
Name: reassignment_method, dtype: int64

In [110]:
df.dtypes

order_date              datetime64[ns]
rider_id                         int64
first_mile_distance            float64
last_mile_distance             float64
alloted_orders                 float64
delivered_orders               float64
cancelled                        int64
undelivered_orders             float64
lifetime_order_count           float64
reassignment_method             object
reassignment_reason             object
reassigned_order               float64
session_time                   float64
cancelled_time                  object
accept_sec                     float64
order_sec                      float64
allot_sec                      float64
year                             int64
month                            int64
dtype: object

In [111]:
num_col = [c for i, c in enumerate(df.columns) if df.dtypes[i] in [np.int, np.float]]
len(num_col)

15

In [112]:
num_col = [c for i, c in enumerate(df.columns) if df.dtypes[i] in [np.object]]
len(num_col)

3

In [113]:
df.isnull().sum()

order_date                   0
rider_id                     0
first_mile_distance          0
last_mile_distance           0
alloted_orders           16948
delivered_orders         17341
cancelled                    0
undelivered_orders       17341
lifetime_order_count        53
reassignment_method     436256
reassignment_reason     436247
reassigned_order        436247
session_time              3675
cancelled_time          444782
accept_sec              449995
order_sec               449995
allot_sec               449995
year                         0
month                        0
dtype: int64

In [114]:
df_test.isnull().sum()

order_date                   0
rider_id                     0
first_mile_distance          0
last_mile_distance           0
alloted_orders            4773
delivered_orders          4884
undelivered_orders        4884
lifetime_order_count       778
reassignment_method     140212
reassignment_reason     140209
reassigned_order        140209
session_time              5054
accept_sec              144839
order_sec               144839
allot_sec               144839
year                         0
month                        0
dtype: int64

In [115]:
df['reassignment_reason'] = df['reassignment_reason'].replace(np.nan, 0)


In [116]:
df_test['reassignment_reason'] = df_test['reassignment_reason'].replace(np.nan, 0)

In [117]:
df['reassignment_reason_freq'] = df['reassignment_reason'].replace(('Auto Reassignment basis Inaction. coreengine.tasks.repush_order_to_aa_bucket','Reassignment Request from SE portal.','Reassign','0'),(7212,5300,1241,436256))
df_test['reassignment_reason_freq'] = df_test['reassignment_reason'].replace(('Auto Reassignment basis Inaction. coreengine.tasks.repush_order_to_aa_bucket','Reassignment Request from SE portal.','Reassign','0'),(2593,1582,460,140209))

In [118]:
df_test['reassignment_reason'].value_counts()

0                                                                               140209
Auto Reassignment basis Inaction. coreengine.tasks.repush_order_to_aa_bucket      2593
Reassignment Request from SE portal.                                              1582
Reassign                                                                           460
Name: reassignment_reason, dtype: int64

In [119]:
pip install --upgrade category_encoders

In [122]:
from category_encoders import TargetEncoder

encoder = TargetEncoder()
df['reassignment_reason']= encoder.fit_transform(df['reassignment_reason'], df['cancelled'])


In [123]:
df.columns

Index(['order_date', 'rider_id', 'first_mile_distance', 'last_mile_distance',
       'alloted_orders', 'delivered_orders', 'cancelled', 'undelivered_orders',
       'lifetime_order_count', 'reassignment_method', 'reassignment_reason',
       'reassigned_order', 'session_time', 'cancelled_time', 'accept_sec',
       'order_sec', 'allot_sec', 'year', 'month', 'reassignment_reason_freq'],
      dtype='object')

In [124]:
df['reassignment_method'] = df['reassignment_method'].replace(np.nan, 0)


In [125]:
df_test['reassignment_method'] = df_test['reassignment_method'].replace(np.nan, 0)

In [126]:
df_test['reassignment_method'].value_counts()

0         140212
auto        4531
manual       101
Name: reassignment_method, dtype: int64

In [127]:
df['reassignment_method_freq'] = df['reassignment_reason'].replace(('auto','manual','0'),(13383,361,436256))
df_test['reassignment_method_freq'] = df_test['reassignment_reason'].replace(('auto','manual','0'),(4531,101,140212))

In [128]:
encoder = TargetEncoder()
df['reassignment_method']= encoder.fit_transform(df['reassignment_method'], df['cancelled'])


In [129]:
df['total_distance'] = df['first_mile_distance'] + df['last_mile_distance']
df_test['total_distance'] = df_test['first_mile_distance'] + df_test['last_mile_distance']

In [130]:
df.columns

Index(['order_date', 'rider_id', 'first_mile_distance', 'last_mile_distance',
       'alloted_orders', 'delivered_orders', 'cancelled', 'undelivered_orders',
       'lifetime_order_count', 'reassignment_method', 'reassignment_reason',
       'reassigned_order', 'session_time', 'cancelled_time', 'accept_sec',
       'order_sec', 'allot_sec', 'year', 'month', 'reassignment_reason_freq',
       'reassignment_method_freq', 'total_distance'],
      dtype='object')

In [131]:
df = df.drop(columns=['reassignment_method1','reassignment_reason1','cancelled_time'],axis =1)

KeyError: ignored

In [132]:
df.head()

,order_date,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,cancelled,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time,cancelled_time,accept_sec,order_sec,allot_sec,year,month,reassignment_reason_freq,reassignment_method_freq,total_distance
0,2021-01-26,11696,1.5666,2.65,46.0,46.0,0,0.0,621.0,0.010313,0.010306,NaN,NaN,NaN,9.0,33.0,24.0,2021,1,0,0.010306,4.2166
1,2021-01-26,18117,2.5207,2.76,8.0,8.0,0,0.0,105.0,0.010313,0.010306,NaN,3.266667,NaN,48.0,89.0,41.0,2021,1,0,0.010306,5.2807
2,2021-01-26,18623,2.2074,4.80,1.0,1.0,0,0.0,66.0,0.010313,0.010306,NaN,9.816667,NaN,16.0,24.0,8.0,2021,1,0,0.010306,7.0074
3,2021-01-26,15945,2.1894,6.38,1.0,1.0,0,0.0,127.0,0.010313,0.010306,NaN,17.533333,NaN,41.0,73.0,32.0,2021,1,0,0.010306,8.5694
4,2021-01-26,17589,2.7870,4.01,34.0,34.0,0,0.0,84.0,0.010313,0.010306,NaN,1.350000,NaN,36.0,87.0,51.0,2021,1,0,0.010306,6.7970
